### 대리점의 매출 이력과 고객 정보 데이터를 이용한 데이터 가공

- 대리점 데이터는 <u>시스템에 의해 관리되는 쇼핑몰 사이트</u>와는 다르게 사람의 손을 탑니다.               
그래서 날짜 입력 실수나 데이터 누락 등'오류'가 많이 포함되어 있습니다.             
이런 **'지저분한' 데이터를 다루는 연습으로 대리점 데이터가 적당**하다고 할 수 있습니다.               
              
              
- 대리점 이외의 비즈니스 현장에서도 엑셀(Excel)처럼 수작업으로 입력하는 경우가 많습니다.                 
손으로 입력한 데이터는 기계적인 입력 확인이 이루어지지 않기 때문에           
데이터가 점점 '지저분'해지고 그대로는 데이터 분석에 활용할 수 없습니다.                   
                    
                   
- 그 외에도, 각각 다른 부서에서 수집된 데이터를 다루는 경우에는                
각기 독자적인 시스템으로 데이터가 관리되므로 그것들을 통일하는 것 또한 쉽지 않습니다.            

고객의 소리)
우리 회사에서는 고객 정보를 엑셀로 관리합니다. 장사는 잘되고 대리점의 매출도 안정적입니다. 데이터가 풍부할 때 데이터 분석을 해두면 여러 가지 발견을 할 수 있지 않을까 합니다. 시험 삼아 데이터 분석을 부탁드립니다.

전제 조건)

상품 : A ~ Z 까지 26개의 상품                         
데이터 : 매출 이력 & 고객 정보 데이터를 담당 사원이 시스템에 '직접 입력'                         

 * 집계 기간 중 상품 단가 변동 X                
 * 매출 이력은 시스템에서 csv파일로 출력                          
 * 고객 정보는 대리점 '관리자가 주별로 집계'해서 엑셀로 관리
<n>

</n>

 uriage.csv : 매출 이력(기간은 2019.01 ~ 2019.07)                   
 kokyaku_daicho.xlsx : 대리점에서 관리하는 고객 정보

In [102]:
import pandas as pd

sale_df = pd.read_csv('C:/Users/LGCARE/Desktop/개인/Study_DataTechniques/01. DataProcess/data/2장/uriage.csv')
customer_df = pd.read_excel('C:/Users/LGCARE/Desktop/개인/Study_DataTechniques/01. DataProcess/data/2장/kokyaku_daicho.xlsx',
                            )

# sale_df.info()
print(sale_df.head())
print()

# customer_df.info()
print(customer_df.head())

# item_name, item_price에 결측치, 오류 존재 > '데이터의 정합성에 문제가 있다'


      purchase_date item_name  item_price customer_name
0  2019-06-13 18:02       상품A       100.0           김가온
1  2019-07-13 13:05     상 품 S         NaN           김우찬
2  2019-05-11 19:42     상 품 a         NaN           김유찬
3  2019-02-12 23:40       상품Z      2600.0           김재현
4   2019-04-22 3:09       상품a         NaN           김강현

   고객이름  지역                  등록일
0  김 현성  H시  2018-01-04 00:00:00
1  김 도윤  E시                42782
2  김 지한  A시  2018-01-07 00:00:00
3  김 하윤  F시                42872
4  김 시온  E시                43127


> 데이터 오류의 예

- 날짜 
1) 2019-10-10
2) 2019/10/10
3) 10/10/2019
4) 2019년 10월 10일                 
: 같은 날짜라도 포맷이 다르면 다른 문자열 데이터가 됩니다.                               
  이런 오류를 자동으로 수정해주는 언어도 있지만 여러 가지 포맷이 섞여 있는 경우에는 주의해야 합니다.

이름
1) 김현성
2) 김 현성
3) 김현 성
4) 김 현성
5) 김현  성                    
: 공백 유무로 시스템은 다른 데이터로 인식합니다.                   
  입력 시 실수로 인해 같은 이름이 다른 이름이 됩니다.

-> 데이터의 오류를 해소하고 정합성을 보장하는 것은 데이터 분석의 기초가 되는 중요한 부분              
-> 정합성을 갖추기 위해서는 먼저 '데이터의 속성'이나 '의미'를 이해해야 합니다.              
   매출 이력에 purchase_date, item_name, item_price, customer_name이 잇는 것을 확인할 수 있습니다.                
   정합성의 확보는 우선 데이터의 오류를 파악하는 것부터 시작합니다.                  

In [103]:
sale_df['item_name'].head() # 공백, 대소문자
sale_df['item_price'].head() # 결측치

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

>> 오류가 있는 상태에서 상품별 월 매출 집계

In [104]:
sale_df['purchase_date'] = pd.to_datetime(sale_df['purchase_date'])
sale_df.dtypes

sale_df['purchase_month'] = sale_df['purchase_date'].dt.strftime('%Y%m')
sale_df['purchase_month'].head()

# 1) groupby
print(sale_df.groupby(['purchase_month', 'item_name'])[['item_price']].sum())

# 2) pivot_table
# size : 상품의 판매 건수
print(pd.pivot_table(data= sale_df, index= 'purchase_month', columns= 'item_name', aggfunc= 'size',
                     fill_value= 0)) # 'size' > values (X)

                          item_price
purchase_month item_name            
201901          상 품 n         1400.0
               상 품H            800.0
               상 품O           1500.0
               상 품P           1600.0
               상 품T           2000.0
...                              ...
201907         상품X           38400.0
               상품Y           27500.0
               상품i             900.0
               상품o            1500.0
               상품r            1800.0

[265 rows x 1 columns]
item_name        상 품 n   상품 E   상품 M   상품 P   상품 S   상품 W   상품 X   상품W  상  품O  \
purchase_month                                                                  
201901               1      0      0      0      0      0      0     0      0   
201902               0      0      0      0      0      0      1     0      0   
201903               0      1      1      1      0      0      0     0      0   
201904               0      0      0      0      0      0      0     1      0   
201905   

In [105]:
# 상품 S와 상품 s가 별도의 상품으로 집계
# 원래 26개의 상품이 99개의 상품으로 늘어남

print(pd.pivot_table(data= sale_df, values= 'item_price', index= 'purchase_month', columns= 'item_name', aggfunc= 'sum',
                     fill_value= 0))

item_name        상 품 n   상품 E    상품 M    상품 P    상품 S    상품 W    상품 X     상품W  \
purchase_month                                                                  
201901          1400.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0   
201902             0.0    0.0     0.0     0.0     0.0     0.0  2400.0     0.0   
201903             0.0  500.0  1300.0  1600.0     0.0     0.0     0.0     0.0   
201904             0.0    0.0     0.0     0.0     0.0     0.0     0.0  2300.0   
201905             0.0    0.0     0.0     0.0  1900.0     0.0     0.0     0.0   
201906             0.0    0.0     0.0     0.0     0.0  2300.0     0.0     0.0   
201907             0.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0   

item_name       상  품O   상  품Q  ...     상품k     상품l     상품o     상품p     상품r  \
purchase_month                 ...                                           
201901            0.0     0.0  ...  1100.0  1200.0  1500.0     0.0     0.0   
201902            0.0     0.0  ...  

>> 상품명 오류 수정

In [106]:
# 명목형

# sale_df.head() # 공백, 대소문자

# 현황 파악

sale_df['item_name'].unique()

# sale_df['item_name'] = sale_df['item_name'].apply(str.upper())
sale_df['item_name'] = sale_df['item_name'].str.upper()
sale_df['item_name'].head()

sale_df['item_name'] = sale_df['item_name'].str.replace(' ', '')
sale_df['item_name'].head()

sale_df['item_name'].unique()
# len(sale_df['item_name'].unique())
sale_df['item_name'].nunique()

sale_df.sort_values(by= 'item_name', ascending= True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


>> 금액 결측치 수정

In [107]:
# 연속형

sale_df['item_price'].head()
sale_df['item_price'].describe()
sale_df.isnull().any(axis= 0)


# 이번 케이스의 경우 집계 기간에 상품 단가의 변동이 없다는 전제 조건이 있었기 때문에
# 결측치는 같은 상품의 단가를 이용하면 수정할 수 있습니다.


purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [108]:
import numpy as np

item_name_price_dict = dict()

item_name_price_dict['상품A']  = int(np.float64(10000))
item_name_price_dict

{'상품A': 10000}

In [ ]:
item_name_price_dict = dict()

for idx, item_nm in enumerate(sale_df['item_name']) :
  # print(idx, ':', item_nm) 
  if pd.notna(sale_df.loc[idx, 'item_price']) :
    item_name_price_dict[item_nm] = int(sale_df.loc[idx, 'item_price'])
  
for idx, item_nm in enumerate(sale_df['item_name']) :
  if pd.isna(sale_df.loc[idx, 'item_price']) :
    sale_df.loc[idx, 'item_price'] = item_name_price_dict[item_nm]

sale_df.isnull().any(axis= 0)
sale_df['item_price'].head()

In [120]:
is_null_df = sale_df['item_price'].isnull()

len(sale_df.loc[is_null_df, 'item_name'].unique())


25

In [ ]:
# 책)

for nan_item_nm in list(sale_df.loc[is_null_df, 'item_name'].unique()) :
  print(nan_item_nm)/
  # price = sale_df.loc[(~is_null_df) & (sale_df['item_name'] == trg), 'item_price'].max()
  # sale_df['item_price'].loc[(is_null_df) & (sale_df['item_name'] == trg)] = price

sale_df.head()

상품S
상품A
상품P
상품N
상품W
상품R
상품I
상품L
상품F
상품O
상품B
상품C
상품V
상품Q
상품U
상품K
상품T
상품X
상품E
상품M
상품G
상품J
상품D
상품H
상품Y


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,NaN,김우찬,201907
2,2019-05-11 19:42:00,상품A,NaN,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,NaN,김강현,201904
